In [1]:
using JuMP, GLPK

Source: https://brainbashers.com/tents.asp


<img src="example.png" style="width: 300px;"/>

In [2]:
# from html, 0=blank, 1=tree, 2=tent
lcpuzzle = "001202011200021001020012000120001200020100100000010200200000201000001212102020000000000010021012201000000200102002020102000001010001001201201212"
lcrownumbers = "332123222404"
lccolnumbers = "222322230505";

Define the problem

In [10]:
puzzle = [parse(Int, i) for i in lcpuzzle]
n = Int(sqrt(length(puzzle)))

trees = [(i, j) for i in 1:n, j in 1:n if puzzle[(i-1)*n + j] == 1]
row_sums = [parse(Int, i) for i in lcrownumbers]
col_sums = [parse(Int, i) for i in lccolnumbers];

Initialize the solver

In [11]:
model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[1:n, 1:n], Bin);

Add constraint for a tent to be next to a tree (but not on a tree)

In [12]:
# a tent must be next to a tree
for row in 1:n, col in 1:n
    # is not on a tree
    if (row, col) in trees
        @constraint(model, x[row, col] == 0)
    end
    
    # should be next to a tree 
    found = false
    for tree in trees
        if abs(tree[1] - row) + abs(tree[2] - col) <= 1
            found = true
        end
    end
    if !found
        @constraint(model, x[row, col] == 0)
    end
end

Add row and column sums

In [13]:
# columns and sums should add up
for i in 1:n
    @constraint(model, sum(x[i, :]) == row_sums[i])
    @constraint(model, sum(x[:, i]) == col_sums[i])
end

Tents cannot touch

In [14]:
# every 2x2 square contains at most 1 tree
for row in 1:(n-1), col in 1:(n-1)
    @constraint(model, sum(x[row:row+1, col:col+1]) <= 1)
end

Solve

In [25]:
JuMP.optimize!(model)

solution = JuMP.value.(x)

for row in 1:n
    for col in 1:n
        if solution[row, col] == 1
            print("⛺")
        elseif (row, col) in trees
            print("🌲")
        else
            print("◽")
        end
        print(" ")
    end
    println("")
end

◽ ◽ 🌲 ⛺ ◽ ⛺ ◽ 🌲 🌲 ⛺ ◽ ◽ 
◽ ⛺ 🌲 ◽ ◽ 🌲 ◽ ⛺ ◽ ◽ 🌲 ⛺ 
◽ ◽ ◽ 🌲 ⛺ ◽ ◽ ◽ 🌲 ⛺ ◽ ◽ 
◽ ⛺ ◽ 🌲 ◽ ◽ 🌲 ◽ ◽ ◽ ◽ ◽ 
◽ 🌲 ◽ ⛺ ◽ ◽ ⛺ ◽ ◽ ◽ ◽ ◽ 
⛺ ◽ 🌲 ◽ ◽ ◽ ◽ ◽ 🌲 ⛺ 🌲 ⛺ 
🌲 ◽ ⛺ ◽ ⛺ ◽ ◽ ◽ ◽ ◽ ◽ ◽ 
◽ ◽ ◽ ◽ 🌲 ◽ ◽ ⛺ 🌲 ◽ 🌲 ⛺ 
⛺ ◽ 🌲 ◽ ◽ ◽ ◽ ◽ ◽ ⛺ ◽ ◽ 
🌲 ◽ ⛺ ◽ ◽ ⛺ ◽ ⛺ ◽ 🌲 ◽ ⛺ 
◽ ◽ ◽ ◽ ◽ 🌲 ◽ 🌲 ◽ ◽ ◽ 🌲 
◽ ◽ 🌲 ⛺ ◽ 🌲 ⛺ ◽ 🌲 ⛺ 🌲 ⛺ 
